In [1]:
import numpy as np
import random

# 2.1 Сформировать порождающую матрицу линейного кода (7, 4, 3).
def Generate_linear_code_matrix(k, n, d):
    I = np.identity(k, dtype=int)  # Создаем единичную матрицу размером k
    X = np.zeros((k, n - k), dtype=int)  # Создаем матрицу нулей для проверки четности

    for i in range(k):
        row = np.random.randint(0, 2, size=n - k)
        while np.sum(row) < d - 1:
            row = np.random.randint(0, 2, size=n - k)
        X[i] = row

    uniq_row = np.unique(X, axis=0)
    while len(uniq_row) < k:
        X = np.zeros((k, n - k), dtype=int)
        for i in range(k):
            row = np.random.randint(0, 2, size=n - k)
            while np.sum(row) < d - 1:
                row = np.random.randint(0, 2, size=n - k)
            X[i] = row
        uniq_row = np.unique(X, axis=0)

    G = np.concatenate((I, X), axis=1)  # Объединяем матрицу идентичности и матрицу проверки четности
    return G

In [2]:
# 2.2 Сформировать проверочную матрицу на основе порождающей.
def Generate_parity_check_matrix(G):
    k, n = G.shape
    H = np.zeros((n - k, n), dtype=int)
    H[:, :k] = G[:, k:].T  # Первые n-k столбцов из транспонированных оставшихся столбцов G
    H[:, k:] = np.identity(n - k, dtype=int)  # Остальные n-k столбцов будут единичной матрицей
    return H

In [3]:
def Generate_syndrome_table(H):

    n = H.shape[1]  # Длина кодового слова
    syndrome_table = {}

    # Однократные ошибки
    for error_position in range(n):
        e = np.zeros(n, dtype=int)
        e[error_position] = 1
        syndrome = np.mod(e @ H.T, 2)
        syndrome_tuple = tuple(syndrome)
        syndrome_table[syndrome_tuple] = [e.tolist()]  # Добавляем список с одним вектором ошибки

    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            e = np.zeros(n, dtype=int)
            e[i] = 1
            e[j] = 1
            syndrome = np.mod(e @ H.T, 2)
            syndrome_tuple = tuple(syndrome)
            if syndrome_tuple in syndrome_table:
                syndrome_table[syndrome_tuple].append(e.tolist())  # Добавляем вектор ошибки в существующий список
            else:
                syndrome_table[syndrome_tuple] = [e.tolist()]  # Создаем новый список с вектором ошибки

    return syndrome_table

In [4]:
# 2.4 Сформировать кодовое слово длины n из слова длины k.
def Encode_message(G, message):
    return np.mod(message @ G, 2)

In [5]:

# Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов.
def Introduce_single_error(codeword, position):
    error_vector = np.zeros_like(codeword)
    error_vector[position] = 1
    return np.mod(codeword + error_vector, 2), error_vector  # Возвращаем слово с ошибкой и вектор ошибки

In [6]:
# Исправление ошибки по синдрому
def Correct_error(received_codeword, H, syndrome_table):
    syndrome = np.mod(received_codeword @ H.T, 2)
    syndrome_tuple = tuple(syndrome)

    if syndrome_tuple in syndrome_table:
        error_vector = syndrome_table[syndrome_tuple]
        corrected_codeword = np.mod(received_codeword + error_vector, 2)
        return corrected_codeword, True

    return received_codeword, False

In [7]:
# 2.5 Сформировать кодовое слово длины n из слова длины k.
def Test_double_error_correction(G, message, syndrome_table):
    codeword = Encode_message(G, message)
    error_positions = np.random.choice(len(codeword), 2, replace=False)

    for pos in error_positions:
        codeword[pos] = 1 - codeword[pos]  # Вносим двукратные ошибки

    syndrome = np.mod(codeword @ H.T, 2)
    return syndrome

In [8]:
# Сгенерировать матрицу линейного кода (7, 4, 3)
G = Generate_linear_code_matrix(4, 7, 3)
print("Матрица G:")
print(G)

Матрица G:
[[1 0 0 0 1 1 0]
 [0 1 0 0 1 0 1]
 [0 0 1 0 1 1 1]
 [0 0 0 1 0 1 1]]


In [9]:
H = Generate_parity_check_matrix(G)
print("\nПроверочная матрица H:")
print(H)


Проверочная матрица H:
[[1 1 1 0 1 0 0]
 [1 0 1 1 0 1 0]
 [0 1 1 1 0 0 1]]


In [10]:
# Генерация таблицы синдромов
syndrome_table = Generate_syndrome_table(H)
print("\nТаблица синдромов:")
for syndrome, error_vector in syndrome_table.items():
    print(f"Синдром: {syndrome}, Вектор ошибки: {error_vector}")


Таблица синдромов:
Синдром: (1, 1, 0), Вектор ошибки: [[1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 0, 1, 1, 0]]
Синдром: (1, 0, 1), Вектор ошибки: [[0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 1]]
Синдром: (1, 1, 1), Вектор ошибки: [[0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 0]]
Синдром: (0, 1, 1), Вектор ошибки: [[0, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 1]]
Синдром: (1, 0, 0), Вектор ошибки: [[0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 1], [0, 0, 1, 1, 0, 0, 0]]
Синдром: (0, 1, 0), Вектор ошибки: [[0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 0, 0], [0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 1]]
Синдром: (0, 0, 1), Вектор ошибки: [[0, 0, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 1, 0]]


In [11]:
message = np.array([1, 0, 1, 1])  # Сообщение (кодовое слово длины k)
codeword = Encode_message(G, message)
print("\nКодовое слово без ошибок:")
print(codeword)


Кодовое слово без ошибок:
[1 0 1 1 0 1 0]


In [12]:
received_codeword, error_vector = Introduce_single_error(codeword, 2)  # Вносим ошибку в 2-й позиции
print("\nКодовое слово с однократной ошибкой:")
print(received_codeword)


Кодовое слово с однократной ошибкой:
[1 0 0 1 0 1 0]


In [ ]:
corrected_codeword, success = Correct_error(received_codeword, H, syndrome_table)
print("\nИсправленное кодовое слово после исправления ошибки:")
print(corrected_codeword)

In [13]:
# Пример использования для 2.5
message = np.array([1, 0, 1, 1])  # Сообщение (кодовое слово длины k)
codeword = Encode_message(G, message)
print("\nКодовое слово без ошибок для двух ошибок:")
print(codeword)


Кодовое слово без ошибок для двух ошибок:
[1 0 1 1 0 1 0]


In [14]:
syndrome = Test_double_error_correction(G, message, syndrome_table)
print("\nСиндром для двукратной ошибки:")
print(syndrome)


Синдром для двукратной ошибки:
[1 0 0]


In [15]:
# Попытка исправления ошибки
corrected_word, success = Correct_error(received_codeword, H, syndrome_table)
print("\nПопытка исправления:", corrected_word)



Попытка исправления: [[1 0 1 1 0 1 0]
 [0 0 0 1 0 1 1]
 [1 1 0 1 0 0 0]
 [1 0 0 0 1 1 0]]


In [23]:
import numpy as np
import random

# Параметры кода
k, n = 5, 13

# 2.6 Сформируем порождающую матрицу линейного кода (n, k, 5).
# Инициализация порождающей матрицы
G = np.zeros((k, n), dtype=int)
for i in range(k):
    G[i, i] = 1

# Исходная матрица X
X = np.array([
    [1, 1, 1, 1, 1, 0, 0, 1],
    [1, 1, 0, 0, 0, 1, 1, 1],
    [1, 0, 1, 1, 0, 1, 0, 1],
    [0, 1, 1, 0, 1, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 1]
])

for i in range(k):
    for j in range(k, n):
        G[i, j] = X[i, j-k]

print("Порождающая матрица G:")
print(G)

# 2.7 Сформируем проверочную матрицу на основе порождающей.
# Создаем матрицу H
H = np.zeros((n - k, n), dtype=int)

# Заполняем нижнюю часть матрицы H
for i in range(n - k):
    H[i, k + i] = 1  # Печатаем единицы для нижней правой части

# Заполняем левую часть матрицы H
for i in range(k):
    H[:, i] = np.roll(X[i], -i)[:n-k]

print("\nПроверочная матрица H:")
print(H)

# Функция для генерации таблицы синдромов
def Generate_syndrome_table(H):

    n = H.shape[1]  # Длина кодового слова
    syndrome_table = {}

    # Однократные ошибки
    for error_position in range(n):
        e = np.zeros(n, dtype=int)
        e[error_position] = 1
        syndrome = np.mod(e @ H.T, 2)
        syndrome_tuple = tuple(syndrome)
        syndrome_table[syndrome_tuple] = [e.tolist()]

    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            e = np.zeros(n, dtype=int)
            e[i] = 1
            e[j] = 1
            syndrome = np.mod(e @ H.T, 2)
            syndrome_tuple = tuple(syndrome)
            if syndrome_tuple in syndrome_table:
                syndrome_table[syndrome_tuple].append(e.tolist())
            else:
                syndrome_table[syndrome_tuple] = [e.tolist()]

    return syndrome_table

# Генерация и присвоение матрицы синдромов
syndrome_table = Generate_syndrome_table(H)

print("Таблица синдромов:")
for syndrome, errors in syndrome_table.items():
    print(f"Синдром: {syndrome}, Ошибки: {errors}")

# Закодировать сообщение
u = np.array([1, 0, 1, 1, 0])
print("\nИсходное сообщение:", u)

v = np.mod(u @ G, 2)
print("Закодированное сообщение:", v)

# Ввести случайную ошибку в кодовое слово
error = np.zeros(n, dtype=int)
error[random.randint(0, n-1)] = 1
print("Введенная ошибка:", error)

v_error = np.mod(v + error, 2)
print("Принятое сообщение с ошибкой:", v_error)

# Вычислить синдром
syndrome = np.mod(v_error @ H.T, 2)
print("Синдром:", syndrome)

# Декодировать полученное слово
syndrome_tuple = tuple(syndrome)
if syndrome_tuple in syndrome_table:
    error_position = syndrome_table[syndrome_tuple][0][0]  # Предполагаем, что это первая позиция ошибки
else:
    print("Ошибка: Синдром не найден в таблице синдромов.")
    error_position = None

if error_position is not None:
    error_vector = np.zeros(n, dtype=int)
    error_vector[error_position] = 1
    print("Вектор ошибки:", error_vector)

    v_corrected = np.mod(v_error + error_vector, 2)
    print("Исправленное сообщение:", v_corrected)

    # Сравнить исправленное сообщение с исходным
    if np.array_equal(v_corrected[:k], u):
        print("Исправленное сообщение совпадает с исходным!")
    else:
        print("Ошибка: Исправленное сообщение не совпадает с исходным!")


Порождающая матрица G:
[[1 0 0 0 0 1 1 1 1 1 0 0 1]
 [0 1 0 0 0 1 1 0 0 0 1 1 1]
 [0 0 1 0 0 1 0 1 1 0 1 0 1]
 [0 0 0 1 0 0 1 1 0 1 0 1 1]
 [0 0 0 0 1 0 0 0 1 1 1 1 1]]

Проверочная матрица H:
[[1 1 1 0 1 1 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 1 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 1 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 1 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 1 0 0]
 [0 1 1 1 0 0 0 0 0 0 0 1 0]
 [1 1 0 1 1 0 0 0 0 0 0 0 1]]
Таблица синдромов:
Синдром: (1, 1, 1, 1, 1, 0, 0, 1), Ошибки: [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]]
Синдром: (1, 0, 0, 0, 1, 1, 1, 1), Ошибки: [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Синдром: (1, 1, 0, 1, 0, 1, 1, 0), Ошибки: [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Синдром: (0, 1, 0, 1, 1, 0, 1, 1), Ошибки: [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Синдром: (1, 1, 1, 1, 0, 0, 0, 1), Ошибки: [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]
Синдром: (1, 0, 0, 0, 0, 0, 0, 0), Ошибки: [[0, 

In [99]:
import numpy as np
import random

# Параметры кода
k, n = 5, 13

# 2.6 Сформируем порождающую матрицу линейного кода (n, k)
# Инициализация порождающей матрицы
G = np.zeros((k, n), dtype=int)
for i in range(k):
    G[i, i] = 1

# Исходная матрица X
X = np.array([
    [1, 1, 1, 1, 1, 0, 0, 1],
    [1, 1, 0, 0, 0, 1, 1, 1],
    [1, 0, 1, 1, 0, 1, 0, 1],
    [0, 1, 1, 0, 1, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 1]
])

for i in range(k):
    for j in range(k, n):
        G[i, j] = X[i, j - k]

print("Порождающая матрица G:")
print(G)

# Сформируем проверочную матрицу на основе порождающей
H = np.zeros((n - k, n), dtype=int)

# Заполняем нижнюю часть матрицы H
for i in range(n - k):
    H[i, k + i] = 1  # Печатаем единицы для нижней правой части

# Заполняем левую часть матрицы H
for i in range(k):
    H[:, i] = np.roll(X[i], -i)[:n - k]

print("\nПроверочная матрица H:")
print(H)

# Функция для генерации таблицы синдромов
def Generate_syndrome_table(H):
    n = H.shape[1]
    syndrome_table = {}

    # Однократные ошибки
    for error_position in range(n):
        e = np.zeros(n, dtype=int)
        e[error_position] = 1
        syndrome = np.mod(e @ H.T, 2)
        syndrome_tuple = tuple(syndrome)
        syndrome_table[syndrome_tuple] = [e.tolist()]

    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            e = np.zeros(n, dtype=int)
            e[i] = 1
            e[j] = 1
            syndrome = np.mod(e @ H.T, 2)
            syndrome_tuple = tuple(syndrome)
            if syndrome_tuple in syndrome_table:
                syndrome_table[syndrome_tuple].append(e.tolist())
            else:
                syndrome_table[syndrome_tuple] = [e.tolist()]

    # Добавляем трехкратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            for l in range(j + 1, n):
                e = np.zeros(n, dtype=int)
                e[i] = 1
                e[j] = 1
                e[l] = 1
                syndrome = np.mod(e @ H.T, 2)
                syndrome_tuple = tuple(syndrome)
                if syndrome_tuple in syndrome_table:
                    syndrome_table[syndrome_tuple].append(e.tolist())
                else:
                    syndrome_table[syndrome_tuple] = [e.tolist()]

    return syndrome_table

# Генерация и присвоение матрицы синдромов
syndrome_table = Generate_syndrome_table(H)

print("\nТаблица синдромов:")
for syndrome, errors in syndrome_table.items():
    print(f"Синдром: {syndrome}, Ошибки: {errors}")

# Закодировать сообщение
u = np.array([1, 0, 1, 1, 0])
print("\nИсходное сообщение:", u)

v = np.mod(u @ G, 2)
print("Закодированное сообщение:", v)

# Ввести трехкратную ошибку в кодовое слово
error_positions = random.sample(range(n), 3)  # Генерируем 3 уникальные позиции для ошибок
error = np.zeros(n, dtype=int)
for pos in error_positions:
    error[pos] = 1
print("Введенные ошибки:", error)

v_error = np.mod(v + error, 2)
print("Принятое сообщение с ошибками:", v_error)

# Вычислить синдром
syndrome = np.mod(v_error @ H.T, 2)
print("Синдром:", syndrome)

# Декодировать полученное слово
syndrome_tuple = tuple(syndrome)
if syndrome_tuple in syndrome_table:
    # Предполагаем, что исправление ошибки возможно
    print("Синдром доступен для исправления.")

    # Попробуем исправить все возможные ошибки
    for error_vector in syndrome_table[syndrome_tuple]:
        error_vector = np.array(error_vector)
        v_corrected = np.mod(v_error + error_vector, 2)

        if np.array_equal(v_corrected[:k], u):
            print("Исправленное сообщение совпадает с исходным!")
            break
    else:

        print("Ошибка: ни одно из исправленных сообщений не совпадает с исходным.")
else:
    print("Ошибка: Синдром не найден в таблице синдромов.")


if np.array_equal(v_error, v):
    print("Исправленное сообщение совпадает с полученным сообщением (ошибка).")
else:
    print("Исправленное сообщение отличается от отправленного, что верно.")


Порождающая матрица G:
[[1 0 0 0 0 1 1 1 1 1 0 0 1]
 [0 1 0 0 0 1 1 0 0 0 1 1 1]
 [0 0 1 0 0 1 0 1 1 0 1 0 1]
 [0 0 0 1 0 0 1 1 0 1 0 1 1]
 [0 0 0 0 1 0 0 0 1 1 1 1 1]]

Проверочная матрица H:
[[1 1 1 0 1 1 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 1 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 1 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 1 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 1 0 0]
 [0 1 1 1 0 0 0 0 0 0 0 1 0]
 [1 1 0 1 1 0 0 0 0 0 0 0 1]]

Таблица синдромов:
Синдром: (1, 1, 1, 1, 1, 0, 0, 1), Ошибки: [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]]
Синдром: (1, 0, 0, 0, 1, 1, 1, 1), Ошибки: [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]]
Синдром: (1, 1, 0, 1, 0, 1, 1, 0), Ошибки: [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]]
Синдром: (0, 1, 0, 1, 1, 0, 1, 1), Ошибки: [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]
Синдром: (1, 1, 1, 1, 0, 0, 0, 1), Ошибки: [[0, 0,

In [57]:
import numpy as np
import random

# Параметры кода
k, n = 5, 13

G = np.zeros((k, n), dtype=int)
for i in range(k):
    G[i, i] = 1

# Исходная матрица X
X = np.array([
    [1, 1, 1, 1, 1, 0, 0, 1],
    [1, 1, 0, 0, 0, 1, 1, 1],
    [1, 0, 1, 1, 0, 1, 0, 1],
    [0, 1, 1, 0, 1, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 1]
])

for i in range(k):
    for j in range(k, n):
        G[i, j] = X[i, j-k]

print("Порождающая матрица G:")
print(G)

# Сформируем проверочную матрицу на основе порождающей.
H = np.zeros((n - k, n), dtype=int)

for i in range(n - k):
    H[i, k + i] = 1

for i in range(k):
    H[:, i] = np.roll(X[i], -i)[:n-k]

print("\nПроверочная матрица H:")
print(H)

# Функция для генерации таблицы синдромов
def Generate_syndrome_table(H):
    n = H.shape[1]  # Длина кодового слова
    syndrome_table = {}

    # Однократные ошибки
    for error_position in range(n):
        e = np.zeros(n, dtype=int)
        e[error_position] = 1
        syndrome = np.mod(e @ H.T, 2)
        syndrome_tuple = tuple(syndrome)
        syndrome_table[syndrome_tuple] = [e.tolist()]  # Добавляем список с одним вектором ошибки

    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            e = np.zeros(n, dtype=int)
            e[i] = 1
            e[j] = 1
            syndrome = np.mod(e @ H.T, 2)
            syndrome_tuple = tuple(syndrome)
            if syndrome_tuple in syndrome_table:
                syndrome_table[syndrome_tuple].append(e.tolist())  # Добавляем вектор ошибки в существующий список
            else:
                syndrome_table[syndrome_tuple] = [e.tolist()]  # Создаем новый список с вектором ошибки

    return syndrome_table

import numpy as np
import random

# Параметры кода
k, n = 5, 13

# Сформируем порождающую матрицу линейного кода (n, k).
# Инициализация порождающей матрицы
G = np.zeros((k, n), dtype=int)
for i in range(k):
    G[i, i] = 1

# Исходная матрица X
X = np.array([
    [1, 1, 1, 1, 1, 0, 0, 1],
    [1, 1, 0, 0, 0, 1, 1, 1],
    [1, 0, 1, 1, 0, 1, 0, 1],
    [0, 1, 1, 0, 1, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 1]
])

for i in range(k):
    for j in range(k, n):
        G[i, j] = X[i, j - k]

print("Порождающая матрица G:")
print(G)

# Сформируем проверочную матрицу на основе порождающей.
H = np.zeros((n - k, n), dtype=int)

# Заполняем нижнюю часть матрицы H
for i in range(n - k):
    H[i, k + i] = 1

# Заполняем левую часть матрицы H
for i in range(k):
    H[:, i] = np.roll(X[i], -i)[:n - k]

print("\nПроверочная матрица H:")
print(H)

# Функция для генерации таблицы синдромов
def Generate_syndrome_table(H):
    n = H.shape[1]  # Длина кодового слова
    syndrome_table = {}

    # Однократные ошибки
    for error_position in range(n):
        e = np.zeros(n, dtype=int)
        e[error_position] = 1
        syndrome = np.mod(e @ H.T, 2)
        syndrome_tuple = tuple(syndrome)
        syndrome_table[syndrome_tuple] = [e.tolist()]
    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            e = np.zeros(n, dtype=int)
            e[i] = 1
            e[j] = 1
            syndrome = np.mod(e @ H.T, 2)
            syndrome_tuple = tuple(syndrome)
            if syndrome_tuple in syndrome_table:
                syndrome_table[syndrome_tuple].append(e.tolist())
            else:
                syndrome_table[syndrome_tuple] = [e.tolist()]

    # Трехкратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            for l in range(j + 1, n):
                e = np.zeros(n, dtype=int)
                e[i] = 1
                e[j] = 1
                e[l] = 1
                syndrome = np.mod(e @ H.T, 2)
                syndrome_tuple = tuple(syndrome)
                if syndrome_tuple in syndrome_table:
                    syndrome_table[syndrome_tuple].append(e.tolist())
                else:
                    syndrome_table[syndrome_tuple] = [e.tolist()]

    return syndrome_table

# Генерация и присвоение матрицы синдромов
syndrome_table = Generate_syndrome_table(H)

print("\nТаблица синдромов:")
for syndrome, errors in syndrome_table.items():
    print(f"Синдром: {syndrome}, Ошибки: {errors}")

# Закодировать сообщение
u = np.array([1, 0, 1, 1, 0])
print("\nИсходное сообщение:", u)

v = np.mod(u @ G, 2)
print("Закодированное сообщение:", v)

# Ввести трехкратную ошибку в кодовое слово
error_positions = random.sample(range(n), 3)  # Генерируем 3 уникальные позиции для ошибок
error = np.zeros(n, dtype=int)
for pos in error_positions:
    error[pos] = 1
print("Введенные ошибки:", error)

v_error = np.mod(v + error, 2)
print("Принятое сообщение с ошибками:", v_error)

# Вычислить синдром
syndrome = np.mod(v_error @ H.T, 2)
print("Синдром:", syndrome)

# Декодировать полученное слово
syndrome_tuple = tuple(syndrome)
if syndrome_tuple in syndrome_table:
    # Предполагаем, что исправление ошибки возможно
    corrections = syndrome_table[syndrome_tuple]

    # Корректировать ошибки
    for error_vector in corrections:
        error_vector = np.array(error_vector)
        v_corrected = np.mod(v_error + error_vector, 2)

        if np.array_equal(v_corrected[:k], u):
            print("Исправленное сообщение совпадает с исходным!")
            break
    else:

        print("Все исправления не совпадают с исходным сообщением.")
else:
    print("Ошибка: Синдром не найден в таблице синдромов.")





Порождающая матрица G:
[[1 0 0 0 0 1 1 1 1 1 0 0 1]
 [0 1 0 0 0 1 1 0 0 0 1 1 1]
 [0 0 1 0 0 1 0 1 1 0 1 0 1]
 [0 0 0 1 0 0 1 1 0 1 0 1 1]
 [0 0 0 0 1 0 0 0 1 1 1 1 1]]

Проверочная матрица H:
[[1 1 1 0 1 1 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 1 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 1 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 1 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 1 0 0]
 [0 1 1 1 0 0 0 0 0 0 0 1 0]
 [1 1 0 1 1 0 0 0 0 0 0 0 1]]
Порождающая матрица G:
[[1 0 0 0 0 1 1 1 1 1 0 0 1]
 [0 1 0 0 0 1 1 0 0 0 1 1 1]
 [0 0 1 0 0 1 0 1 1 0 1 0 1]
 [0 0 0 1 0 0 1 1 0 1 0 1 1]
 [0 0 0 0 1 0 0 0 1 1 1 1 1]]

Проверочная матрица H:
[[1 1 1 0 1 1 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 1 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 1 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 1 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 1 0 0]
 [0 1 1 1 0 0 0 0 0 0 0 1 0]
 [1 1 0 1 1 0 0 0 0 0 0 0 1]]

Таблица синдромов:
Синдром: (1, 1, 1, 1, 1, 0, 0, 1), Ошибки: [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]]
Си